# 1. Generate TEST and TRAINING dataframes

In [8]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from operator import itemgetter 
from sklearn.grid_search import GridSearchCV 
from sklearn import tree, cross_validation, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Import TRAINING dataframe
train_df = pd.read_csv("./training_df_without_standardization.csv", sep="\t", header=0)

feature_labels = ['height', 'hips', 'waist', 'dress', 'shoes', 
                  'insta_num_post', 'insta_avg_likes', 'insta_avg_comments', 'agencies']
train_indep = train_df[feature_labels]
train_dep = []
fshows = list(train_df.fashionshows)
for i in fshows:
    if i < 1: train_dep.append(0)
    else: train_dep.append(1)


In [2]:
#Import TEST dataframe
test_df = pd.read_csv("/Users/ferrarae/Dropbox/PAPERS/Jaehyuk_Park/ipynb/test_df_without_standardization.csv", sep="\t", header=0)
test_indep = test_df[['height', 'hips', 'waist', 'dress', 'shoes', 'insta_post', 'insta_likes', 'insta_comments', 'agencies']]
test_dep = list(test_df.fashionshows)
print test_indep

    height  hips  waist  dress  shoes  insta_post  insta_likes  \
0      178  86.5     58     33   41.0          59          148   
1      178  86.0     60     33   40.0          24           32   
2      179  88.0     61     34   39.0           0            0   
3      180  89.0     60     34   41.0          52           93   
4      175  86.0     58     33   38.0         163           70   
5      180  89.0     60     34   41.0           2           48   
6      180  90.0     61     34   40.0          10          116   
7      178  87.0     61     33   39.0          34           90   
8      183  86.0     62     34   41.0          16           61   
9      176  87.0     59     33   38.5          17          647   
10     177  86.0     60     32   38.5          38           51   
11     180  90.0     60     33   40.0          29           59   
12     169  88.0     60     33   38.0          49          570   
13     179  94.0     65     35   41.0          58           52   
14     180

# 2. Testing with three classifiers

In [ ]:
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

def get_top_n_clfs_params(grid_search, n_top=3):
    top_scores = sorted(grid_search.grid_scores_, key=itemgetter(1), reverse=True)[:n_top]
    clfs_parameters = list()
    for i, score in enumerate(top_scores): clfs_parameters.append(score.parameters)
    return clfs_parameters

In [ ]:
mlClassifier = [tree.DecisionTreeClassifier(), 
                ensemble.RandomForestClassifier(), # n_jobs=-1
                ensemble.AdaBoostClassifier()] 

# dt = mlClassifier[0]
erf = mlClassifier[1]
# eab = ensemble.AdaBoostClassifier() # mlClassifier[0], n_estimators=100
# adaboost = ensemble.AdaBoostClassifier(base_estimator= mlClassifier[1])
# lr = LogisticRegression() # C=.01

# base_clf = BernoulliNB(alpha=.001, binarize=0.0, class_prior=None, fit_prior=False) 
# base_clf = mlClassifier[0]
# eab = ensemble.AdaBoostClassifier(base_estimator=erf)
#indep_df = mlModel[md_count]

x_train = train_indep.loc[:]
y_train = train_dep

x_test = test_indep.loc[:]
y_test = test_dep

# use a full grid over all parameters
param_grid = {"n_estimators": [10, 20, 35, 50, 75],
              "max_depth": [3, 4, None],
              "max_features": [3, 5, 7, 9],
              "min_samples_split": [1, 3, 5, 7, 9],
              "min_samples_leaf": [3, 5, 7, 9],
              "bootstrap": [True, False],
              "oob_score": [True, False],
              "criterion": ["gini", "entropy"]}

grid_search = GridSearchCV(erf, param_grid=param_grid) # n_jobs=-1, cv=None
grid_search.fit(x_train, y_train)
report(grid_search.grid_scores_)

In [ ]:
# model generation

niter = 1000
n_top_clfs = 10
# print grid_search.best_params_
# erf = grid_search.best_estimator_ # ensemble.RandomForestClassifier()

# dt_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# eab_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# elr_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

clfs_parameters = get_top_n_clfs_params(grid_search, n_top_clfs)

clfs = list()

for params in clfs_parameters:
    rf = ensemble.RandomForestClassifier(n_estimators = params['n_estimators'], max_depth = params['max_depth'], max_features = params['max_features'],
                                         min_samples_split = params['min_samples_split'],  min_samples_leaf = params['min_samples_leaf'],
                                         bootstrap = params['bootstrap'], oob_score = params['oob_score'], criterion = params['criterion'])
    clfs.append(rf)

# predictions
for erf in clfs:
    print '\n\n',erf
    erf_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    feature_selection = [.0, .0, .0, .0, .0, .0, .0, .0, .0]

    for i in range(niter):

    #     dt.fit(x_train, y_train)
        erf.fit(x_train, y_train)
    #     eab.fit(x_train, y_train)
    #     lr.fit(x_train, y_train)
    #     boost.fit(x_train, y_train)

        importances = erf.feature_importances_ 
        std = np.std([tre.feature_importances_ for tre in erf.estimators_], axis=0)
        indices = np.argsort(importances)[::-1]
    #     print("Feature ranking:")

        for f in range(9):
    #         print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
            feature_selection[indices[f]] += importances[indices[f]]

    #     dt_pred = dt.predict(x_test)
        erf_pred = erf.predict(x_test)
    #     eab_pred = eab.predict(x_test)
    #     elr_pred = lr.predict(x_test)
    #     eboo_pred = boost.predict(x_test)

    #     dt_result = [float(x + y) for x, y in zip(dt_result, dt_pred)]
        erf_result = [float(x + y) for x, y in zip(erf_result, erf_pred)]
    #     eab_result = [float(x + y) for x, y in zip(eab_result, eab_pred)]
    #     elr_result = [float(x + y) for x, y in zip(elr_result, elr_pred)]
    #     eboo_result = [x + y for x, y in zip(eboo_result, eboo_pred)]

    # dt_result = [x / niter for x in dt_result]
    erf_result = [x / niter for x in erf_result]
    # eab_result = [x / niter for x in eab_result]
    # elr_result = [x / niter for x in elr_result]

    # eboo_result = [(x*1.0) / niter for x in eboo_result]
    fs_result = [x / niter for x in feature_selection]

    print y_test
    # print dt_result, roc_auc_score(y_test, dt_result)
    print erf_result, roc_auc_score(y_test, erf_result)
    # print eab_result, roc_auc_score(y_test, eab_result)
    # print elr_result, roc_auc_score(y_test, elr_result)
    # print eboo_result, roc_auc_score(y_test, eboo_result) # eboo_result[:, 1]

    # print fs_result
    print("Feature importance:")
    for f in range(9):
        print("%s (%f)" % (feature_labels[f], fs_result[f]))

In [ ]:
# alternative AUROC calculation
fpr, tpr, thresholds = roc_curve(y_test, erf_result)
roc_auc = auc(fpr, tpr)

# Without standardized features

Model with rank: 1
Mean validation score: 0.779 (std: 0.029)
Parameters: {'bootstrap': True, 'min_samples_leaf': 9, 'n_estimators': 50, 'min_samples_split': 1, 'criterion': 'gini', 'max_features': 3, 'max_depth': 4}

Model with rank: 2
Mean validation score: 0.775 (std: 0.027)
Parameters: {'bootstrap': True, 'min_samples_leaf': 3, 'n_estimators': 50, 'min_samples_split': 5, 'criterion': 'gini', 'max_features': 7, 'max_depth': 3}

Model with rank: 3
Mean validation score: 0.775 (std: 0.001)
Parameters: {'bootstrap': True, 'min_samples_leaf': 7, 'n_estimators': 50, 'min_samples_split': 7, 'criterion': 'entropy', 'max_features': 7, 'max_depth': 3}

[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0.103, 0.011, 0.0, 0.022, 0.0, 0.152, 0.069, 0.195, 0.02, 0.116, 0.01, 0.0, 0.0, 0.0, 0.011] 0.8125

Feature importance:
height (0.156474)
hips (0.089913)
waist (0.100431)
dress (0.054049)
shoes (0.085814)
insta_num_post (0.162151)
insta_avg_likes (0.175071)
insta_avg_comments (0.128604)
agencies (0.047493)

# With standardized features

Model with rank: 1
Mean validation score: 0.779 (std: 0.021)
Parameters: {'bootstrap': True, 'min_samples_leaf': 9, 'n_estimators': 35, 'min_samples_split': 1, 'criterion': 'entropy', 'max_features': 3, 'max_depth': 4}

Model with rank: 2
Mean validation score: 0.779 (std: 0.005)
Parameters: {'bootstrap': True, 'min_samples_leaf': 9, 'n_estimators': 75, 'min_samples_split': 9, 'criterion': 'entropy', 'max_features': 5, 'max_depth': 4}

Model with rank: 3
Mean validation score: 0.775 (std: 0.011)
Parameters: {'bootstrap': True, 'min_samples_leaf': 7, 'n_estimators': 20, 'min_samples_split': 7, 'criterion': 'gini', 'max_features': 7, 'max_depth': 4}

[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0.049, 0.0, 0.0, 0.0, 0.0, 0.001, 0.006, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.6875

Feature importance:
height (0.256350)
hips (0.051115)
waist (0.051732)
dress (0.036158)
shoes (0.022353)
insta_num_post (0.118131)
insta_avg_likes (0.168837)
insta_avg_comments (0.129382)
agencies (0.165942)

In [10]:
int_eboo =[round(x) for x in eboo_result]
print int_eboo
print classification_report(y_test, int_eboo)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
             precision    recall  f1-score   support

          0       0.47      1.00      0.64         7
          1       0.00      0.00      0.00         8

avg / total       0.22      0.47      0.30        15



# 3. Output the result in Latex format

In [11]:
test_indep['runway'] = y_test
test_indep['decision tree'] = dt_result
test_indep['random forest'] = erf_result
test_indep['ada boost'] = eab_result

print test_indep.to_latex(index=True, header=True)

\begin{tabular}{lrrrrrrrrrrrrr}
\toprule
{} &  height &  hips &  waist &  dress &  shoes &  insta\_post &  insta\_likes &  insta\_comments &  agencies &  runway &  decision tree &  random forest &  ada boost \\
\midrule
0  &     178 &  86.5 &     58 &     33 &   41.0 &          59 &          148 &               4 &         1 &       1 &              0 &              0 &        0.1 \\
1  &     178 &  86.0 &     60 &     33 &   40.0 &          24 &           32 &               0 &         0 &       1 &              0 &              0 &        0.7 \\
2  &     179 &  88.0 &     61 &     34 &   39.0 &           0 &            0 &               0 &         1 &       1 &              0 &              0 &        0.0 \\
3  &     180 &  89.0 &     60 &     34 &   41.0 &          52 &           93 &               1 &         1 &       1 &              0 &              0 &        0.0 \\
4  &     175 &  86.0 &     58 &     33 &   38.0 &         163 &           70 &               2 &         1 &    